In [ ]:
from pyapacheatlas.auth import ServicePrincipalAuthentication
from pyapacheatlas.core import PurviewClient
from pyapacheatlas.core.util import GuidTracker
from dotenv import load_dotenv
import os

load_dotenv()

tenant_id = os.environ.get("TENANT_ID") 
client_id = os.environ.get("CLIENT_ID")
client_secret = os.environ.get("CLIENT_SECRET")
account_name = os.environ.get("PURVIEW_ACCOUNT")

auth = ServicePrincipalAuthentication(
    tenant_id = tenant_id, 
    client_id = client_id, 
    client_secret = client_secret
)

# Create a client to connect to your service.
client = PurviewClient(
    account_name = account_name,
    authentication = auth
)

gt = GuidTracker()

In [ ]:
client.get_all_typedefs()

In [ ]:
client.endpoint_url

In [ ]:
client.get_entity(qualifiedName="https://purviewatlaspoc.dfs.core.windows.net/datalake/extended_vaccine_data/{SparkPartitions}", typeName="azure_datalake_gen2_resource_set")

In [ ]:
client.get_relationship(guid='a4021762-cd09-4f75-bf80-ea727e1e13b2')

In [ ]:
client.glossary.get_glossary()

In [ ]:
from pyapacheatlas.core import AtlasClient, AtlasEntity, TypeCategory

# Create a Tabular Schema entity
ts = AtlasEntity(
        name="demotabschema",
        typeName="tabular_schema",
        qualified_name="pyapache://demotabschema",
        guid = -1
    )
    # Create a Column entity that references your tabular schema
col01 = AtlasEntity(
        name="democolumn",
        typeName="column",
        qualified_name="pyapche://mycolumn",
        guid= -2,
        attributes={
            "type":"Blah",
            "description": "blabh blah"
        },
        relationshipAttributes = {
            "composeSchema": ts.to_json(minimum=True)
        }
    )

    # Create a resource set that references the tabular schema
rs = AtlasEntity(
        name="demoresourceset",
        typeName="azure_datalake_gen2_resource_set",
        qualified_name="pyapache://demors",
        guid = -3,
        relationshipAttributes = {
            "tabular_schema": ts.to_json(minimum=True)
        }
    )

In [ ]:
## Get datasources

import requests

client.scan_endpoint_url = "https://purviewatlaspoc.scan.purview.azure.com"
url = client.scan_endpoint_url + \
    f"/datasources"

getDataSources = requests.get(
    url,
    headers =  client.authentication.get_authentication_headers(),
    params = { "api-version": "2018-12-01-preview"}
)

response = client._handle_response(getDataSources)
response

In [ ]:
## Create or update scan
import requests

scan_name = "datalakeScan"
data_source = "AzureDataLakeStorage-ZSB"

client.scan_endpoint_url = "https://purviewatlaspoc.scan.purview.azure.com"
url = client.scan_endpoint_url + \
    f"/datasources/{data_source}/scans/{scan_name}"

createOrUpdateScan = requests.put(
    url,
    headers =  client.authentication.get_authentication_headers(),
    params = { "api-version": "2018-12-01-preview"},
    json = {
    "kind": "AdlsGen2Msi",
    "properties": {
        "scanRulesetName": "AdlsGen2",
        "scanRulesetType": "System",
        "collection": {
            "referenceName": account_name,
            "type": "CollectionReference"
        }
    }
}
)

response = client._handle_response(createOrUpdateScan)
response

In [ ]:
account_name